<a href="https://colab.research.google.com/github/lee-messi/machine-learning/blob/main/disaster_tweets_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow --quiet
!pip install tensorflow-hub --quiet
!pip install tensorflow-text --quiet

/bin/bash: /usr/local/bin/pip: /usr/bin/python: bad interpreter: No such file or directory
/bin/bash: /usr/local/bin/pip: /usr/bin/python: bad interpreter: No such file or directory
/bin/bash: /usr/local/bin/pip: /usr/bin/python: bad interpreter: No such file or directory
/bin/bash: /usr/local/bin/pip: /usr/bin/python: bad interpreter: No such file or directory
/bin/bash: /usr/local/bin/pip: /usr/bin/python: bad interpreter: No such file or directory


In [2]:
import os, re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

2023-07-16 14:31:36.226473: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## Disaster Tweets

In [7]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [8]:
train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

We are going to use the tweet text to predict whether or not the tweet is about an actual disaster.

In [9]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

There are 6,023 sentences with label 1 and 2,528 sentences with label 0. Let's balance these out.

In [10]:
train = train.groupby('target').sample(3000)

In [11]:
train['full_text'] = train.keyword + " " + train.location + " " + train.text
train.full_text = train.full_text.apply(lambda x: str(x))

In [12]:
train_df, val_df = np.split(train.sample(frac = 1), [int(0.8 * len(train))])

In [13]:
def clean_text(dataframe):
    dataframe.full_text.apply(lambda x: x.lower())
    dataframe.full_text.apply(lambda x: re.sub(r'http\S+', '', x))
    dataframe.full_text.apply(lambda x: re.sub(r'\W+', ' ', x))
    dataframe.full_text.apply(lambda x: re.sub(r'\d+', '', x))
    dataframe.full_text.apply(lambda x: x.lower())
    return(dataframe)

In [14]:
cleaned_train = clean_text(train_df)
cleaned_val = clean_text(val_df)

In [15]:
cleaned_train.head(5)

,id,keyword,location,text,target,full_text
1397,2016,casualties,"Bronx, New York",Warfighting Robots Could Reduce Civilian Casua...,1,"casualties Bronx, New York Warfighting Robots ..."
6108,8721,sinking,Liverpool,Do you feel like you are sinking in low self-i...,1,sinking Liverpool Do you feel like you are sin...
5298,7568,outbreak,"Stockholm, Sweden",@SenateMajLdr let's try to do our best to prev...,1,"outbreak Stockholm, Sweden @SenateMajLdr let's..."
358,516,army,NaN,One Direction Is my pick for http://t.co/q2eBl...,1,nan
5778,8245,rioting,Spare 'Oom,if they kill off Val I'm rioting #Emmerdale,0,rioting Spare 'Oom if they kill off Val I'm ri...


## Classifier Model using BERT

In [16]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [17]:
bert_preprocess = hub.KerasLayer(tfhub_handle_preprocess)
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

2023-07-16 14:36:04.029655: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
2023-07-16 14:36:10.039172: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-07-16 14:36:10.039785: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message)

In [18]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string)
encoder_input = bert_preprocess(text_input)
encoder_output = bert_encoder(encoder_input)

l = tf.keras.layers.Dense(100, activation = 'relu')(encoder_output['sequence_output'])
l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences = True, dropout = 0.3))(l)
l = tf.keras.layers.Dense(25, activation = 'relu')(l)
l = tf.keras.layers.Dense(1, activation = 'sigmoid')(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])

2023-07-16 14:47:05.714970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-07-16 14:47:05.757069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-07-16 14:47:06.307400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-07-16 1

In [19]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                  patience = 2)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'model/best_performed_model.ckpt',
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_loss',
    verbose = 1
)

In [27]:
history = model.fit(cleaned_train.full_text,
                    cleaned_train.target,
                    validation_data = (cleaned_val.full_text, cleaned_val.target),
                    epochs = 30,
                    callbacks = [early_stopping, model_checkpoint_callback])

Epoch 1/30


2023-07-16 14:51:08.214083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 14:51:08.216816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 14:51:08.219343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

150/150 [==============================] - ETA: 0s - loss: 0.5884 - accuracy: 0.6586 

2023-07-16 15:26:41.342649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 15:26:41.345200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 15:26:41.348153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 0.55629, saving model to best_performed_model.ckpt
150/150 [==============================] - 2562s 17s/step - loss: 0.5884 - accuracy: 0.6586 - val_loss: 0.5563 - val_accuracy: 0.6765
Epoch 2/30
 76/150 [==============>...............] - ETA: 15:45 - loss: 0.5492 - accuracy: 0.6884

KeyboardInterrupt: 

In [72]:
final = pd.read_csv('drive/MyDrive/Colab Notebooks/disaster-tweets/test.csv')

In [73]:
final['text'] = final['text'].apply(lambda x: re.sub(r'http\S+', '', x))
final['text'] = final['text'].apply(lambda x: re.sub(r'\W+', ' ', x))
final['text'] = final['text'].apply(lambda x: re.sub(r'\d+', '', x))
final['text'] = final['text'].apply(lambda x: x.lower())

In [74]:
predictions = (model.predict(final.text) > 0.5).astype(int).ravel()

102/102 [==============================] - 113s 1s/step


In [75]:
predictions_df = pd.DataFrame(list(zip(final.id, predictions)),
                              columns = ['id', 'target'])

In [76]:
predictions_df.to_csv('predictions.csv', index = False)